In [4]:
import json
import csv
import jsonlines
def load_passage_tsv(tsv_path: str, text_first = True):
    """
    load the tsv of a passage file
    """
    doc_lookup_dict = {}
    with open(tsv_path, "r", 2 ** 20) as f:
        next(f)
        f = csv.reader(f, delimiter='\t', quotechar='"')
        f = (l for l in f if len(l) == 3)
        n = 1
        for i, text, title in tqdm(f):
            if text == title:
                continue
            i = int(i)

            # assert i==n, f"i=={i}, n=={n}"
            # n+=1
            if not text_first:
                text, title = title, text
            doc_lookup_dict[i] = {
                'text': text,
                'title': title
            }

    print("Processed total number of docs {}".format(n))
    return doc_lookup_dict
import os
import csv
from tqdm import tqdm
import json

In [5]:
doc_lookup_dict = load_passage_tsv(os.path.join("/mnt/cache/zhuty/SEAL_data/dataset", "psgs_w100.tsv"))


21015324it [03:05, 113215.16it/s]

Processed total number of docs 1


In [22]:
dump_dict = [{"id": passage_id, "contents": f"{item['title']}\n{item['text']}"} for passage_id, item in doc_lookup_dict.items()]

In [23]:
folder_name = "DPR_orig"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/dense_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/lucene_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/data
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/lucene_output
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/dense_output


In [24]:
import jsonlines

In [25]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
dump_dict = [{"id": passage_id, "contents": f"{item['title']}\n{item['text']}"} for passage_id, item in doc_lookup_dict.items()]
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

In [ ]:
folder_path = os.path.join(BASE_PATH, folder_name)
print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")
print("===== combine index")
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(folder_path, 'dense_index', 'index_shard_')} --shard-num 8")

In [29]:
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(BASE_PATH, 'DPR_orig_no_title', 'dense_index', 'index_shard_')} --shard-num 8")

python -m pyserini.index.merge_faiss_indexes --prefix /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/dense_index/index_shard_ --shard-num 8


#### Orig without title

In [6]:
folder_name = "DPR_orig_no_title"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/dense_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/lucene_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/data
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/lucene_output
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title/dense_output


In [ ]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
dump_dict = [{"id": passage_id, "contents": f"\n {item['text']}"} for passage_id, item in doc_lookup_dict.items()]
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

In [10]:
folder_path = os.path.join(BASE_PATH, folder_name)
print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")



bash scripts_general/construct_lucene_index.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title
bash scripts_general/evaluate_bm25.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title
bash scripts_general/evaluate_dense.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig_no_title 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cac

In [ ]:
python -m pyserini.index.merge_faiss_indexes --prefix /mnt/cache/zhuty/CS6220_data/pyserini_data/DPR_orig/dense_index/index_shard_ --shard-num 8


#### Paragraph


In [25]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path, text_first=False)

32774319it [03:03, 178959.83it/s]


Processed total number of docs 1


In [26]:
folder_name = "paragraph"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph/dense_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph/lucene_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph/data
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph/lucene_output
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph/dense_output


In [27]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
dump_dict = [{"id": passage_id, "contents": f"{item['title']}\n{item['text']}"} for passage_id, item in doc_lookup_dict.items()]
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

In [20]:
folder_path = os.path.join(BASE_PATH, folder_name)
print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")


bash scripts_general/construct_lucene_index.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph
bash scripts_general/evaluate_bm25.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph
bash scripts_general/evaluate_dense.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph 4
conda act

### Paragraph no title

In [ ]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path, text_first=False)

7552289it [00:42, 182778.27it/s]

In [ ]:
folder_name = "paragraph_no_title"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

In [20]:
doc_lookup_dict[3]

{'text': 'He is most famous for making lyrical and artistic films within the context of popular Hindi cinema of the 1950s, and expanding its commercial conventions, starting with his 1957 film "Pyaasa". Several of his later works have a cult following. His movies attract full houses when re-released; especially in Germany, France and Japan.',
 'title': 'Guru Dutt'}

In [21]:
len(doc_lookup_dict)

32774319

In [23]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
dump_dict = [{"id": passage_id, "contents": f"\n{item['text']}"} for passage_id, item in doc_lookup_dict.items()]
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

In [24]:
print("finished")

finished


In [11]:
folder_path = os.path.join(BASE_PATH, folder_name)
print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")
print("===== combine index")
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(folder_path, 'dense_index', 'index_shard_')} --shard-num 8")


bash scripts_general/construct_lucene_index.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title
bash scripts_general/evaluate_bm25.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title
bash scripts_general/evaluate_dense.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/paragraph_no_title 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /

In [ ]:
bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence/data 3

#### Sentence

In [30]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path, text_first=False)

32774319it [03:03, 178170.64it/s]


Processed total number of docs 1


In [31]:
import tempfile

import nltk
def get_sentence_splitter():
  temp_dir = tempfile.mkdtemp()
  nltk.download("punkt", download_dir=temp_dir)
  return nltk.data.load(
      os.path.join(temp_dir, "tokenizers/punkt/english.pickle"))
sentence_spliter = get_sentence_splitter()

[nltk_data] Downloading package punkt to /tmp/tmp1tmzp71t...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [32]:
folder_name = "sentence"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/dense_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/lucene_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/data
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/lucene_output
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/dense_output


In [34]:
dump_dict = []
for passage_id, item in tqdm(doc_lookup_dict.items()):
    curr_sent_idx = 0
    for sentence in sentence_spliter.tokenize(item['text']):
        dump_dict.append({"id": f"{passage_id}_{curr_sent_idx}", "contents": f"{item['title']}\n{sentence}"} )
        curr_sent_idx += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32774319/32774319 [25:07<00:00, 21741.30it/s]


In [39]:
len(dump_dict)

98367012

In [ ]:
dump_dict[0]

In [42]:
dump_dict[300]

{'id': '87_1',
 'contents': 'Hardware overlay\nTypically one of these output devices has to be declared the "primary" device, and only the primary device can display hardware overlays.'}

In [2]:
BASE_PATH='/home/aiops/zhuty/CS6220_data/pyserini_data'

In [48]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

In [49]:
dump_dict[-1]

{'id': '38269407_2',
 'contents': 'Algorithmic Number Theory Symposium\nThe proceedings of ANTS XI and ANTS XII were published as a special issue of the London Mathematical Society Journal of Computation and Mathematics.'}

In [ ]:
dump_dict[0]

In [4]:

import os

In [8]:
folder_name = "sentence"

folder_path = os.path.join(BASE_PATH, folder_name)


print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")
print("===== combine index")
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(folder_path, 'dense_index', 'index_shard_')} --shard-num 8")


bash scripts_general/construct_lucene_index.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence
bash scripts_general/evaluate_bm25.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence
bash scripts_general/evaluate_dense.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence 4
conda act

### Sentence no title

In [10]:
import json
import csv
import jsonlines
def load_passage_tsv(tsv_path: str, text_first = True):
    """
    load the tsv of a passage file
    """
    doc_lookup_dict = {}
    with open(tsv_path, "r", 2 ** 20) as f:
        next(f)
        f = csv.reader(f, delimiter='\t', quotechar='"')
        f = (l for l in f if len(l) == 3)
        n = 1
        for i, text, title in tqdm(f):
            if text == title:
                continue
            i = int(i)

            # assert i==n, f"i=={i}, n=={n}"
            # n+=1
            if not text_first:
                text, title = title, text
            doc_lookup_dict[i] = {
                'text': text,
                'title': title
            }

    print("Processed total number of docs {}".format(n))
    return doc_lookup_dict
import os
import csv
from tqdm import tqdm
import json

In [11]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path, text_first=False)

32774319it [03:07, 174942.23it/s]

Processed total number of docs 1


In [15]:
import tempfile

import nltk
def get_sentence_splitter():
  temp_dir = tempfile.mkdtemp()
  nltk.download("punkt", download_dir=temp_dir)
  return nltk.data.load(
      os.path.join(temp_dir, "tokenizers/punkt/english.pickle"))
sentence_spliter = get_sentence_splitter()

[nltk_data] Downloading package punkt to /tmp/tmp59sbzk90...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /tmp/tmp3dfm2x1j...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
folder_name = "sentence_no_title"
BASE_PATH = "/home/aiops/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title
mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title/dense_index
mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title/lucene_index
mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title/data
mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title/lucene_output
mkdir /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title/dense_output


In [17]:

folder_path = os.path.join(BASE_PATH, folder_name)


print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")
print("===== combine index")
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(folder_path, 'dense_index', 'index_shard_')} --shard-num 8")


bash scripts_general/construct_lucene_index.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title
bash scripts_general/evaluate_bm25.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title
bash scripts_general/evaluate_dense.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /home/aiops/zhuty/CS6220_data/pyserini_data/sentence_no_title 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /

In [18]:
dump_dict = []
for passage_id, item in tqdm(doc_lookup_dict.items()):
    curr_sent_idx = 0
    for sentence in sentence_spliter.tokenize(item['text']):
        dump_dict.append({"id": f"{passage_id}_{curr_sent_idx}", "contents": f"\n{sentence}"} )
        curr_sent_idx += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32774319/32774319 [25:55<00:00, 21076.71it/s]


In [21]:
data_path = os.path.join(BASE_PATH, folder_name, "data", "documents.jsonl")
with jsonlines.open(data_path, 'w') as writer:
    writer.write_all(dump_dict)

### Sentence (Note: This is an duplicate of the previous)

In [4]:
import json
import csv
import jsonlines
def load_passage_tsv(tsv_path: str, text_first = True):
    """
    load the tsv of a passage file
    """
    doc_lookup_dict = {}
    with open(tsv_path, "r", 2 ** 20) as f:
        next(f)
        f = csv.reader(f, delimiter='\t', quotechar='"')
        f = (l for l in f if len(l) == 3)
        n = 1
        for i, text, title in tqdm(f):
            if text == title:
                continue
            i = int(i)

            # assert i==n, f"i=={i}, n=={n}"
            # n+=1
            if not text_first:
                text, title = title, text
            doc_lookup_dict[i] = {
                'text': text,
                'title': title
            }

    print("Processed total number of docs {}".format(n))
    return doc_lookup_dict
import os
import csv
from tqdm import tqdm
import json

In [5]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path, text_first=False)

32774319it [03:00, 181311.68it/s]

Processed total number of docs 1


In [7]:
import tempfile

import nltk
def get_sentence_splitter():
  temp_dir = tempfile.mkdtemp()
  nltk.download("punkt", download_dir=temp_dir)
  return nltk.data.load(
      os.path.join(temp_dir, "tokenizers/punkt/english.pickle"))
sentence_spliter = get_sentence_splitter()

[nltk_data] Downloading package punkt to /tmp/tmp_t7mu1gm...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
folder_name = "sentence"
BASE_PATH = "/mnt/cache/zhuty/CS6220_data/pyserini_data/"
print(f"mkdir {os.path.join(BASE_PATH, folder_name)}")
for name in ['dense_index', 'lucene_index', 'data','lucene_output', 'dense_output']:
    print(f"mkdir {os.path.join(BASE_PATH, folder_name, name)}")

mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/dense_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/lucene_index
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/data
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/lucene_output
mkdir /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence/dense_output


In [10]:

folder_path = os.path.join(BASE_PATH, folder_name)


print(f"""
bash scripts_general/construct_lucene_index.sh {folder_path}
bash scripts_general/evaluate_bm25.sh {folder_path}
bash scripts_general/evaluate_dense.sh {folder_path}

""")

print("================== index")
for i in range(8):
    print(f"conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh {folder_path} {i}")
print("===== combine index")
print(f"python -m pyserini.index.merge_faiss_indexes --prefix {os.path.join(folder_path, 'dense_index', 'index_shard_')} --shard-num 8")


bash scripts_general/construct_lucene_index.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title
bash scripts_general/evaluate_bm25.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title
bash scripts_general/evaluate_dense.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title


================== index
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title 0
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title 1
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title 2
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cache/zhuty/CS6220_data/pyserini_data/sentence_no_title 3
conda activate pyserini-dev; bash scripts_general/construct_dense_index_shard.sh /mnt/cac

In [ ]:
dump_dict = []
for passage_id, item in tqdm(doc_lookup_dict.items()):
    curr_sent_idx = 0
    title = item['title']
    for sentence in sentence_spliter.tokenize(item['text']):
        dump_dict.append({"id": f"{passage_id}_{curr_sent_idx}", "contents": f"{title}\n{sentence}"} )
        curr_sent_idx += 1

## Note: The following code is no longer supported

In [14]:
outpath_lucene = "/mnt/cache/zhuty/CS6220_data/DPR_orig_title_text_pyserini_lucene/docs.json"
json.dump(dump_dict, open(outpath_lucene, "w"))

In [15]:
dump_dict = [{"id": passage_id, "contents": f"{item['text']}"} for passage_id, item in
             doc_lookup_dict.items()]


In [16]:
outpath_lucene = "/mnt/cache/zhuty/CS6220_data/DPR_orig_text_no_space_pyserini_lucene/docs.json"
json.dump(dump_dict, open(outpath_lucene, "w"))

In [9]:

import jsonlines

outpath = "/mnt/cache/zhuty/CS6220_data/DPR_original_pyserini.jsonl"
actual = dump_dict
print(len(actual))
with jsonlines.open(outpath, 'w') as writer:
    writer.write_all(actual)

21015324


In [11]:
outpath_lucene = "/mnt/cache/zhuty/CS6220_data/DPR_orig_pyserin_lucene/docs.json"
json.dump(actual, open(outpath_lucene, "w"))

In [8]:
len(dump_dict)

21015324

In [28]:
import jsonlines

outpath = "/home/aiops/zhuty/CS6220/DPR_original_test_version.json"
actual = dump_dict[:20000]
print(len(actual))
with jsonlines.open(outpath, 'w') as writer:
    writer.write_all(actual)

20000


In [31]:
outpath_lucene = "/home/aiops/zhuty/CS6220/test_DPR_pyserini/docs.json"
json.dump(actual, open(outpath_lucene, "w"))

In [23]:
import jsonlines

outpath = "/home/aiops/zhuty/CS6220/DPR_original_pyserini_version.json"
actual = dump_dict[:]
print(len(actual))
with jsonlines.open(outpath, 'w') as writer:
    writer.write_all(actual)

21015324


### Paragraph related

In [24]:
tsv_path = "/home/aiops/zhuty/CS6220/language/paragraph_blocks_en_wiki_20181220/paragraphs.tsv"
doc_lookup_dict = load_passage_tsv(tsv_path)

38269407it [03:20, 190757.43it/s]


Processed total number of docs 1


In [25]:
dump_dict = [{"id": passage_id, "contents": f"{item['text']}\n"} for passage_id, item in doc_lookup_dict.items()]
outpath = "/home/aiops/zhuty/CS6220/DPR_paragraph_pyserini_version.json"
actual = dump_dict
print(len(actual))
with jsonlines.open(outpath, 'w') as writer:
    writer.write_all(actual)

32774487


In [32]:
string_a = "a\n\nb\n\nc"
string_a.split("\n")

['a', '', 'b', '', 'c']

Note: The rest of the work is moved to explore_wiki_data.ipynb. This is because instead of reading and then loading, we can directly write the paragraphs.